импорты и константы

In [1]:
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
import re
import shutil
import time

# 153.8336486816406250 884.2712384333286764    измеряемый диапазон. 0-2136 диапазон данных
start = 400  # нм
end = 700  # нм
step = (884.2712384333286764 - 153.8336486816406250) / 2136

# номера точек в массиве по координате х
start_point = round((start - 153.8336486816406250) / step)
end_point = start_point + int((end - start) / step)
_start = start_point + 11
_end = start_point + round((end_point - start_point) / 2)

функции получения данных

In [2]:
def get_max_rmr(path, fl):
    _dm = np.zeros(len(fl))

    for i in range(len(fl)):
        with open(path + fl[i], "r", encoding="utf8") as spec:
            _dm[i] = re.split(",", spec.read())[_start + 425]
    return _dm

получение максимумов из спектров

In [17]:
data_max = np.zeros(1)  # изанчальные графики
main_folder = r"C:\Users\Nik\Desktop\оливка\10"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
print(main_folder)
file_list = np.array(natsorted(os.listdir(main_folder)))
print(len(file_list), " файлов")
for i in range(0, len(file_list), 5000):
    _b = i + 5000
    if _b > len(file_list):
        _b = len(file_list)
    data_max = np.append(data_max, get_max_rmr(main_folder, file_list[i:_b]), axis=0)
    print("получено ", _b)
print("получено " + str(len(data_max)) + " максимумов спектров")

C:/Users/Nik/Desktop/оливка/10/
50195  файлов
получено  5000
получено  10000
получено  15000
получено  20000
получено  25000
получено  30000
получено  35000
получено  40000
получено  45000
получено  50000
получено  50195
получено 50196 максимумов спектров


максимумы на таймлайнах

In [18]:
_step = 400
_sp = 0
_ep = _step

# создание папки для сохранения картинок
dist_folder = r"C:\Users\Nik\Desktop\оливка обработка"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"
folder_name = "10 timelines"
if not os.path.exists(os.path.join(dist_folder, folder_name)):
    os.makedirs(os.path.join(dist_folder, folder_name))

# расчет гистограммы
num_bins = 10  # количество интервалов
range_values = (np.max(data_max) - np.min(data_max)) / num_bins
# границы интервалов
bin_edges = np.arange(np.min(data_max), np.max(data_max) + range_values, range_values)
# середины интервалов
bins = bin_edges[:-1] + range_values / 2

hist, _ = np.histogram(data_max, bins=bin_edges)

zero = bin_edges[np.argmax(hist) + 1]
fig, ax = plt.subplots(figsize=(10, 6), dpi=120)
fig.suptitle("гистограмма")
ax.hist(data_max, bin_edges, alpha=0.6, color="blue", edgecolor="k", linewidth=1)
ax.plot(bins, hist, color="red", lw=2)
plt.xticks(rotation=90)
ax.set_xticks(bin_edges)
plt.savefig(dist_folder + "/" + folder_name + "/" + "гистограмма.png")

m = np.min(data_max) * 0.8
m2 = np.max(data_max) * 0.5

for i in range(round(len(data_max) / (3 * _step))):
    fig, axs = plt.subplots(3, 1, figsize=(10, 6), dpi=120)
    plt.ioff()
    _s = _sp
    for j in range(3):
        if _ep > len(data_max):
            _ep = len(data_max)
        axs[j].plot(np.arange(_sp, _ep, 1), data_max[_sp:_ep], lw=1, color="blue")
        axs[j].scatter(np.arange(_sp, _ep, 1), data_max[_sp:_ep], s=4, color="black")
        axs[j].plot((_sp, _ep), (zero, zero), color="orange")
        axs[j].set_ylim(0, 0.5)
        _sp += _step
        _ep += _step

    _e = _ep - _step
    fig.suptitle("спектры " + str(_s) + "-" + str(_e))
    plt.savefig(
        dist_folder
        + "/"
        + folder_name
        + "/"
        + "ряд максимумов "
        + str(_s)
        + "-"
        + str(_e)
        + ".png"
    )

In [7]:
point_index = []
_point = False
k = []
zero = 0.05
s = 500
for i in range(0, len(data_max), s):
    ar = np.mean(data_max[i : i + s])
    if ar > zero:
        k.append(i)
        _point = True
    if ar < zero and _point == True:
        point_index.append((k[0], i))
        k = []
        _point = False
    if _point == True and i >= len(data_max) - s:
        point_index.append((k[0], len(data_max) - 1))
print(len(point_index))
print(point_index)

7
[(10500, 12000), (14500, 16500), (19500, 21500), (24000, 24500), (43000, 44000), (46000, 47500), (49500, 51000)]


In [22]:
for i in range(len(point_index)):
    print(i+1, ' ',point_index[i], "   ", point_index[i][1] - point_index[i][0])

1   [10550, 11550]     1000
2   [15250, 16250]     1000
3   [19900, 20900]     1000
4   [24550, 25550]     1000
5   [29200, 30200]     1000
6   [32500, 33400]     900
7   [35850, 36800]     950
8   [39000, 39900]     900
9   [42250, 43100]     850
10   [45400, 46400]     1000
11   [48700, 49600]     900


In [19]:
point_index = [
[10550,11550],
[15250,16250],
[19900,20900],
[24550,25550],
[29200,30200],
[32500,33400],
[35850,36800],
[39000,39900],
[42250,43100],
[45400,46400],
[48700,49600],
]

In [23]:
dist_folder = r"C:\Users\Nik\Desktop\оливка обработка"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"

folder_name = "10 points"
d_name = "10_"


if os.path.exists(os.path.join(dist_folder, folder_name)):

    shutil.rmtree(dist_folder + "/" + folder_name)


os.makedirs(os.path.join(dist_folder, folder_name))


for i in range(len(point_index)):

    os.makedirs(os.path.join(dist_folder + "/" + folder_name, d_name + str(i + 1)))

In [24]:
main_folder = r"C:\Users\Nik\Desktop\оливка\10"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
file_list = np.array(natsorted(os.listdir(main_folder)))
print(dist_folder)


for i in range(len(point_index)):

    dist = dist_folder + "/" + folder_name + "/" + d_name + str(i + 1)

    for j in range(point_index[i][0], point_index[i][1]):

        shutil.copy2(main_folder + str(file_list[j]), dist)

    print(d_name, i + 1, " ", point_index[i][1] - point_index[i][0])

C:/Users/Nik/Desktop/оливка обработка/
10_ 1   1000
10_ 2   1000
10_ 3   1000
10_ 4   1000
10_ 5   1000
10_ 6   900
10_ 7   950
10_ 8   900
10_ 9   850
10_ 10   1000
10_ 11   900
